In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

On design un chalet en bois, on prend du triple vitrage pour nos 1 fenêtres de 2 m², sur le mur  et pour lisolant on prend de la laine de roche. Pour une pièce de 3x4=12m², donc deux
murs (appelés mur1) de 3m par 3m de hauteur, et deux de 4 par 3 (appelés mur2).

In [ ]:
# thermal conductivity, W/(m·K);
lambdabois=0.06 # wood
lambdaverre=1   # glass
lambdaair=0.024 # air
lambdalaine = 0.034 # whool

# épaisseurs
everre=0.04
elaine=0.25
ebois=0.2

# surfaces
Smur1 = 9
Smur2=12
Sfenetre=2

# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

#convection coefficient, W/m² K;
h_in = 8
h_out = 25 

Ti0 = 10    #temperature outside, K;
Tin = 20    #temperature inside, K;
σ = σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

### Thermal Conductances and capacities


Thermal conductances (W/K)

explication schéma:

VERT :adiabatique pas pris en compte

ROUGE: mur
échanges outdoor T0 = Text
0 : convection ext
1 : long wave raidiation soleil

résistances 1 et 2: bois
capa 1: bois

rés 3, 4 isolant
capa 2 isolant

noeud 4 : temp mur haut int

VERT: verre
t0: Text
R12: cond verre
R13: thermique verre

noeud 7: temp verre (fenêtre) int

noeud 5 : Tint1
R5 : conv mur haut air chambre 1 int
R9 : conv verre air chambre 1 int

noeud 6: T surface mur milieu chambre 1
R8 : résistance rayonnement chambre 1 mur haut mur milieu
R7 : résistance rayonnement chambre 1 fenêtre mur milieu
NB: pas de prise en compte du rayonnement entre les murs à 90°

noeud 8 : Temp mur droite chambre 1
R11 : conv mur droite et int chambre 1
R10 : rés mur droite (porte confondue)

R17 rés mur milieu chambre 1 et 2
R6 : rés mur milieu chambre 1 avec air chambre 1

noeud 10 : Temp mur droite chambre 2
R16 : conv mur milieu et int chambre 2
R15 : conv mur droite et int chambre 2
R14 : rés mur droite chmabre 2 (R14 = R10)

Conductances 7 et 8 are longwave radiation conductances 

In [ ]:
#Conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

#Convection
Gw = h * wall['Surface'][0]     # wall
Gg = h * wall['Surface'][2]     # glass

#Long Wave radiation
Tm = Tin + 273   # K, mean temp for radiative exchange
Fwg = glass['Surface']  / wood['Surface']    # view factor wall-glass

#Between window and wall_3
G7_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G7_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G7_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * glass['Surface']
G7 = 1 / (1 / G7_1 + 1 / G7_12 + 1 / G7_2) 

#Between wall_1 and wall_3
G8_1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * S_wall_3
G8_12 = 4 * σ * Tm**3 * Fwg * S_wall_3 
G8_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * S_wall_1 
G8 = 1 / (1 / G8_1 + 1 / G8_12 + 1 / G8_2)


Thermal capacities

In [ ]:
#Conduction

#Convection

#Long Wave radiation

#Advection

In [ ]:
#Conduction
Rmur1 = 2*ebois/(lambdabois*(Smur2-Sfenetre))+elaine/(lambdalaine*(Smur2-Sfenetre))
Rfenetre = 3*everre/(lambdaverre*Sfenetre)+2*eair/(lamdaair*Sfenetre)
Rmur2 = 2*ebois/(lambdabois*Smur2)+elaine/(lambdalaine*Smur2)

#Convection
Rconv2 = 2/(h*Smur2)
Rconv1 = 2/(h*Smur1)
Rconvfen = 2/(h*Sfenetre)

# long wave radiation
Tm = Tin + 273   # K, mean temp for radiative exchange

GLW1 = 4 * σ * Tm**3 * ε_wLW / (1 - ε_wLW) * wall['Surface']['Layer_in']
GLW12 = 4 * σ * Tm**3 * Fwg * wall['Surface']['Layer_in']
GLW2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * wall['Surface']['Glass']

Gtot = 1 / (1 / GLW1 + 1 / GLW12 + 1 / GLW2)